# Anomaly detection for clickstream data

## SageMaker Random Cut Forest 

In [142]:
%store -r css streams

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import boto3
import botocore
import sagemaker
import sys

In [146]:
streams

,url,clickstream_id,user_session_id
timestamp,,,
2012-03-01 00:00:00,http://www.RL.com/,24,13
2012-03-01 00:10:00,http://www.RL.com/,22,14
2012-03-01 00:20:00,http://www.RL.com/,20,8
2012-03-01 00:30:00,http://www.RL.com/,17,9
2012-03-01 00:40:00,http://www.RL.com/,15,8
...,...,...,...
2012-03-15 23:10:00,http://www.RL.com/video/1003,6,4
2012-03-15 23:20:00,http://www.RL.com/video/1003,3,3
2012-03-15 23:30:00,http://www.RL.com/video/1003,6,5


In [ ]:
from sagemaker import RandomCutForest

session = sagemaker.Session()

# specify general training job information
rcf = RandomCutForest(role=execution_role,
                      train_instance_count=1,
                      train_instance_type='ml.m4.xlarge',
                      data_location='s3://{}/{}/'.format(bucket, prefix),
                      output_path='s3://{}/{}/output'.format(bucket, prefix),
                      num_samples_per_tree=512,
                      num_trees=50)

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(streams.value.to_numpy().reshape(-1,1)))

In [ ]:
rcf_inference = rcf.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
)